In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import re
# os.chdir("E:\\GitHub\\QA-abstract-and-reasoning")
sys.path.append("E:\\GitHub\\QA-abstract-and-reasoning")

# 改进词向量的预处理

In [11]:
from utils.config import WV_MODEL_PAD
from gensim.models import word2vec

## 读取词向量

In [13]:
wv_model = word2vec.Word2Vec.load(WV_MODEL_PAD)

## 问题词发现

In [15]:
wv_model.wv.most_similar(["奔驰"])

[('ML', 0.5625964403152466),
 ('S3', 0.5582177042961121),
 ('GLK', 0.5312835574150085),
 ('宝马', 0.5268276929855347),
 ('C180', 0.5238099098205566),
 ('00L', 0.5044093132019043),
 ('E3', 0.5040868520736694),
 ('保时捷', 0.49885332584381104),
 ('E260', 0.48449334502220154),
 ('S500', 0.47810783982276917)]

In [16]:
wv_model.wv.most_similar(["宝马"])

[('318I', 0.5821380615234375),
 ('525', 0.5632237792015076),
 ('523', 0.5609207153320312),
 ('华晨宝马', 0.5473986864089966),
 ('730LI', 0.5443926453590393),
 ('E90', 0.5348121523857117),
 ('LI', 0.5342251062393188),
 ('奔驰', 0.5268276929855347),
 ('五系', 0.5230212211608887),
 ('525LI', 0.5215471386909485)]

In [19]:
from utils.saveLoader import Vocab
from utils.config import VOCAB_PAD
vocab = Vocab(VOCAB_PAD)

In [43]:
vocab.word_to_id("奔驰")

938

## 追溯回原始文本

In [2]:
from utils.saveLoader import load_dataset, get_text
from utils.config import TRAIN_DATA, TEST_DATA
train_df, test_df = load_dataset(TRAIN_DATA, TEST_DATA)
raw_text = get_text(train_df, test_df)
# 只获取问题和对话列
qd_text = get_text(train_df, columns=['Question', 'Dialogue'])
# 只获取标签列的文本
report_text = get_text(train_df, columns=["Report"])

程序get_text()共耗时1.48秒
程序get_text()共耗时1.13秒
程序get_text()共耗时0.98秒


感觉上这个问题不是很关键

In [67]:
r = re.compile(".{15}ML.{15}", re.S)
r.findall(raw_text)[:10]

['师说：[语音] 随时联系\n奔驰ML500排气凸轮轴调节错误 技师',
 '概多少费用 技师说：你好，奔驰ML350在4s保养一次2000左',
 '|车主说：好，谢谢 你好，奔驰ML350在4s保养一次2000左',
 '师说：[语音] 随时联系\n奔驰ML350空调空气质量传感器在那里',
 '时间看看 很高兴为你解答\n奔驰ML350CDI柴油版可以换美孚1',
 '保险丝烧了，室内灯保险是ROOMLAMP\n宝马 zf 8速变速箱',
 '平时开的比较多。\n11款老款的ML350 可以和手机连蓝牙。但是',
 'TRIP按键就可以复位的\n奔驰ML350柴油发动机报识别到端子8',
 '锐宝XL 6AT 添加了350ML变速箱油添加剂进去  请问还需',
 '来就可以了|车主说：放出350ML  容易控制吗？|技师说：容易']

In [16]:
len(r.findall(report_text))

5317

# 改进
## 删除数字标题

In [7]:
# 删除1. 2. 3. 这些标题
r = re.compile("\D(\d\.)\D")  # re.S
# print("\n".join(r.findall(report_text)))
num_title = r.findall(raw_text)
print("共有：{}个".format(len(num_title), ))
num_title[:5]

共有：5685个


['1.', '2.', '3.', '4.', '5.']

## 整合常出现的词

In [15]:
r = re.compile("根据.{2,4}描述.{10}", re.S)
print("\n".join(r.findall(report_text)))

根据你的描述和交流     这种
根据车主描述，问题应该是有地方泄
根据故障异响描述的声音。判断不会对车
根据你的描述，应该是车辆发动机的
根据客户描述，可能是翻板与发动机
根据客户描述，该车更换离合器三件
根据车主描述，应该是没排干净空气
根据你的描述，以及沟通情况，需要
根据你的描述这是车门闭锁器坏了，
根据你的描述以及沟通情况，可能的
根据你的描述以及沟通情况，车辆做
根据车主描述应该是两个安全带有问
根据你的描述以及沟通情况，结合故
根据车主描述，应该是油泵的油浮子
根据您的描述，建议先检查发电机发
根据你的描述，这是车辆后部的电子
根据你的描述，有可能是变矩器磨损
根据你的描述以及沟通情况，如果线
根据你的描述自以及沟通情况，如果
根据你的描述建议更换遥控电池，如
根据你的描述，可以拆后备箱左右内
根据你的描述以及沟通情况，需要重
根据你的描述，已经确认问题，建议
根据你的描述和问诊的情况，我判断
根据你的描述和故障码的显示，你只
根据你的描述与沟通，请用电脑对气
根据你的描述你的故障需要专用电脑
根据客户描述，需要更换保险杠。

根据客户描述需要检查主气囊以及油
根据客户描述，需要检查节气门以及
根据你的描述，你的故障出在变速器
根据客户描述，该声音为气门与液压
根据客户描述，需要检查一下是否是
根据客户描述，需要检查继电器与伺
根据客户描述，该车平衡轴链条记号
根据客户描述，需要检查排档拉线以
根据客户描述，需要检查水温传感器
根据客户描述需要检查轮胎气压以及
根据客户描述需要检查遥控接收器以
根据你的描述，你的车为喷油系统出
根据你的描述与沟通得知你目前车辆
根据你的描述，应该是车辆发动机存
根据你的描述，出现这个问题需要同
根据您的描述，这个情况是由于高温
根据你的描述，你的这个最大可能是
根据你的描述，你的变速器是没有什
根据你的描述，现在没有办法确定是
根据你的描述，你的发动机系统没有
根据你的描述这种情况，你需要清洗
根据你的描述，我建议你到修理厂把
根据你的描述，这种情况，你已经做
根据你的描述，发动机系统报了一个
根据你的描述，这是车辆通信线有问
根据你的描述以及沟通情况，需要重
根据你的描述，这是变速箱双离合器
根据你的描述，这种情况应该是车辆
根据你的描述，你的爱车为监牙连接
根据您的描述，这个情况的一般原因
根据你的描述，毛病应该在

In [16]:
len(r.findall(report_text))

3265

## 删除汽车大师样本

In [23]:
r = re.compile("\n.*您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！.*\n")
r.findall(report_text)

['\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n不可以！您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n如图您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下载汽车大师APP，在APP内搜索我的名字，关注后，即可与我一对一图文或电话沟通，我会尽力为您解决问题！\n',
 '\n您当前的问题已经解决，建议您下

## 删除qd常出现report不常出现的词

In [3]:
from collections import Counter
from utils.config import USER_DICT, STOP_WORDS
import jieba
jieba.load_userdict(USER_DICT)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Light\AppData\Local\Temp\jieba.cache
Loading model cost 0.740 seconds.
Prefix dict has been built succesfully.


In [4]:
# 删掉非汉字词
s = "集团啊ad243.-手+/动dd阀**得分#adf"
r = re.compile("[^\u4e00-\u9fa5]")
r.sub("", s)

'集团啊手动阀得分'

In [5]:
qd = r.sub("", qd_text)
report = r.sub("", report_text)
qd_words = jieba.cut(qd)
report_words = jieba.cut(report)

In [6]:
qd_words_count = Counter(qd_words)
print("1/2")
report_words_count = Counter(report_words)

In [45]:
sum(report_words_count.values())

1475850

In [40]:
compare = [ (qd[0], qd[1], rep[1]) for qd in qd_words_count.most_common()[:1000] 
           for rep in report_words_count.most_common() if qd[0]==rep[0] and rep[1]<=1000]
         

In [41]:
compare

[('车主说', 513053, 6),
 ('吗', 106917, 323),
 ('语音', 73984, 10),
 ('吧', 34402, 536),
 ('谢谢', 34067, 26),
 ('现在', 30897, 729),
 ('怎么', 27485, 67),
 ('啊', 27003, 188),
 ('嗯', 25441, 62),
 ('呢', 24211, 162),
 ('能', 24154, 934),
 ('多少', 22908, 198),
 ('那个', 20508, 810),
 ('跑', 20296, 961),
 ('请问', 17967, 30),
 ('师傅', 17880, 508),
 ('哦', 17420, 140),
 ('感觉', 16650, 297),
 ('开', 16462, 605),
 ('知道', 16128, 252),
 ('年', 15286, 343),
 ('又', 14990, 179),
 ('他', 13732, 472),
 ('款', 12614, 382),
 ('着', 12094, 588),
 ('钱', 11761, 231),
 ('大师', 11726, 107),
 ('个', 11006, 529),
 ('熄火', 10661, 515),
 ('不客气', 10308, 2),
 ('出来', 10005, 627),
 ('两个', 9980, 1000),
 ('想', 9864, 177),
 ('买', 9859, 360),
 ('得', 9446, 648),
 ('关系', 9434, 977),
 ('哪里', 9429, 297),
 ('档', 9428, 660),
 ('跟', 9402, 503),
 ('一直', 9317, 441),
 ('自己', 9308, 672),
 ('找', 9048, 888),
 ('加', 8994, 713),
 ('加油', 8832, 589),
 ('之前', 8807, 305),
 ('其他', 8748, 872),
 ('踩', 8617, 446),
 ('有点', 8572, 338),
 ('起来', 8556, 611),
 ('肯定', 8501, 828

In [46]:
new_stop_words = [w[0] for w in compare if w[1]>10000]

In [65]:
from utils.preprocess import load_stop_words
stop_words = load_stop_words(STOP_WORDS)

In [67]:
# with open(STOP_WORDS, mode="a", encoding="utf-8") as f:
#     for word in new_stop_words:
#         if word not in stop_words:
#             f.write(word+"\n")

In [58]:
s1 = "kfkwe4中1.55午样子8342.111,什么7438282,akdfke1w4"
s2 = "dk4545edfkke不在55意的样子-834.2111,dsamskw45s"

# 匹配汉字后面的数字 包括小数点，负号
r = re.compile("[\u4e00-\u9fa5]([\d|\d.+|\-\d]+)")
r.findall(s2)

['55', '-834.2111']